
# <center><font color= #F08080 > <b>CU25_Modelo de gestión de Lista de Espera Quirúrgica</font></center>

#  <font color='green'>IV. Model development</font>

En este anexo se incluye el código utilizado durante el desarrollo 
de los modelos incluidos en el caso de uso.

## Modelo DES - proceso masivo



In [1]:
Sys.setlocale(category = "LC_ALL", locale = "es_ES.UTF-8")

[1] "es_ES.UTF-8/es_ES.UTF-8/es_ES.UTF-8/C/es_ES.UTF-8/C"

### Paquetes

In [2]:
## PROCESO PREDICCIÓN Y SIMULACIÓN MASIVA


library(readr)
library(tidyverse)
library(tidymodels)
library(lubridate)
library(modeltime)
library(simmer)
library(simmer.plot)
library(tidyr)

## EN CASO DE QUE LA PREDICCIÓN AL FINAL DEL HORIZONTE RESULTE EN UN NÚMERO NEGATIVO, SE TOMA EL
## ÚLTIMO VALOR POSITIVO (O LO QUE ES LO MISMO, SE ELIMINAN LOS NEGATIVOS)



── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ dplyr   1.1.0
✔ tibble  3.2.1     ✔ stringr 1.5.0
✔ tidyr   1.3.0     ✔ forcats 0.5.2
✔ purrr   1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.3     ✔ rsample      1.1.1
✔ dials        1.1.0     ✔ tune         1.0.1
✔ infer        1.0.4     ✔ workflows    1.1.3
✔ modeldata    1.1.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.4     ✔ yardstick    1.1.0
✔ recipes      1.0.4     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks readr::spec()
✖ rec

## Función

In [3]:
#s especialidad y zona
#periodo
#h horizonte para predicción serie
params_sim <- function(s, periodo, h){
  pred_h_personas <- modelo_pacientes |> 
    pluck(s) |>
    modeltime_forecast(h = h) |> 
    mutate(id = "personas")
  
  pred_h_tiempo <- modelo_tiempo |> 
    pluck(s) |>
    modeltime_forecast(h = h) |> 
    mutate(id = "tiempo")
  
  if(periodo == "ultimo"){
    
    pacientes_en_cola <- modelo_pacientes |> 
      pluck(s, ".calibration_data", 1) |> 
      slice_max(fecha) |> 
      pull(.actual)
    
    tiempo_medio_en_cola <- modelo_tiempo |> 
      pluck(s, ".calibration_data", 1) |> 
      slice_max(fecha) |> 
      pull(.actual)
    
    
    
  } else{
    
    pacientes_en_cola <- pred_h_personas |> 
      filter(.value > 0) |> 
      slice_tail(n = 1) |> pull(.value)
    
    tiempo_medio_en_cola <- pred_h_tiempo |> 
      filter(.value > 0) |> 
      slice_tail(n = 1) |> pull(.value)
  }
  
  if(pacientes_en_cola > 0 & tiempo_medio_en_cola > 0){
    lambda <- pacientes_en_cola / tiempo_medio_en_cola
    mu <- lambda/pacientes_en_cola
  } else{
    lambda <- mu <- NA
  }
  
  return(list(pacientes_en_cola = pacientes_en_cola,
              tiempo_medio_en_cola = tiempo_medio_en_cola,
              lambda = lambda,
              mu = mu,
              pred_h_tiempo = pred_h_tiempo,
              pred_h_personas = pred_h_personas))
}



### Datos

In [4]:

## Datos y modelos

modelo_pacientes <- read_rds("modelos_pacientes_xgboost.rds")
modelo_tiempo <- read_rds("modelos_tiempo_xgboost.rds")

capacidad <- read_csv("CU_25_05_07_01_capacidad.csv")

variables <- read_csv("VARIABLES.csv") 

df <- data.frame(s = names(modelo_pacientes)) |> 
  tidyr::separate(s, c("nombre_area", "Especialidad"), "\\.", remove = FALSE) |> 
  left_join(capacidad)


Rows: 160 Columns: 4
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): id_area, nombre_area, Especialidad
dbl (1): capacidad

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 2 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (7): variable, tipo, control, nombre, etiqueta, descripcion, instrucciones
dbl (3): valor, min, max

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Joining with `by = join_by(nombre_area, Especialidad)`


## Simulación

In [5]:



## COGER DE VARIABLES !!
H <- variables |> 
  filter(variable == "HORIZONTE") |> 
  pull(valor) |> 
  as.numeric()

# H <- 2
NPER <- variables |> 
  filter(variable == "NPER") |> 
  pull(valor)

#simulación
message("Simulación según último valor")
l_sim_ultimo <- df |> 
  # slice(1:2) |>
  pull(s) |> 
  map(~{
    pars <- params_sim(s = .x, periodo = "ultimo", h = H)
    cap <- df |> filter(s == .x) |> 
      pull(capacidad)
    env <- simmer("listasSim")
    
    paciente <- trajectory("Trayectoria del paciente") %>%
      ## Operación
      seize("quirofano", 1) %>%
      timeout(function() rexp(1, pars$mu)) %>%
      release("quirofano", 1)
    
    env %>%
      add_resource("quirofano", cap) %>%
      add_generator("inicial", paciente, at(rep(0, pars$pacientes_en_cola))) |>
      add_generator("paciente", paciente, function() rpois(1, pars$lambda))
    
    env %>%
      run(NPER)
    
    recursos <- get_mon_resources(env)
    llegadas <- get_mon_arrivals(env, ongoing = TRUE)
    return(list(recursos = recursos |> mutate(id = .x),
                llegadas = llegadas |> mutate(id = .x),
                pred_h_personas = pars$pred_h_personas |> mutate(id = .x),
                pred_h_tiempo = pars$pred_h_tiempo |> mutate(id = .x)))
    
  }, .progress = TRUE) 
message("Simulación según horizonte")
l_sim_h <- df |> 
  # slice(1:2) |>
  pull(s) |> 
  map(~{
    pars <- params_sim(.x, periodo = "pred", h = H)
    cap <- df |> filter(s == .x) |> 
      pull(capacidad)
    env <- simmer("listasSim")
    
    paciente <- trajectory("Trayectoria del paciente") %>%
      ## Operación
      seize("quirofano", 1) %>%
      timeout(function() rexp(1, pars$mu)) %>%
      release("quirofano", 1)
    
    env %>%
      add_resource("quirofano", cap) %>%
      add_generator("inicial", paciente, at(rep(0, pars$pacientes_en_cola))) |>
      add_generator("paciente", paciente, function() rpois(1, pars$lambda))
    
    env %>%
      run(NPER)
    
    recursos <- get_mon_resources(env)
    llegadas <- get_mon_arrivals(env, ongoing = TRUE)
    return(list(recursos = recursos |> mutate(id = .x),
                llegadas = llegadas |> mutate(id = .x),
                pred_h_personas = pars$pred_h_personas |> mutate(id = .x),
                pred_h_tiempo = pars$pred_h_tiempo |> mutate(id = .x)))
    
  }, .progress = TRUE) 




sim_ultimo_recursos <- l_sim_ultimo |> map_dfr(~.x |> pluck("recursos"))
sim_ultimo_llegadas <- l_sim_ultimo |> map_dfr(~.x |> pluck("llegadas"))

sim_h_recursos <- l_sim_h |> map_dfr(~.x |> pluck("recursos"))
sim_h_llegadas <- l_sim_h |> map_dfr(~.x |> pluck("llegadas"))

pred_h_tiempo <- l_sim_h |> map_dfr(~.x |> pluck("pred_h_tiempo"))
pred_h_personas <- l_sim_h |> map_dfr(~.x |> pluck("pred_h_personas"))


write_rds(sim_ultimo_recursos, "sim_ultimo_recursos.rds")
write_rds(sim_ultimo_llegadas, "sim_ultimo_llegadas.rds")
write_rds(sim_h_recursos, "sim_h_recursos.rds")
write_rds(pred_h_tiempo, "pred_h_tiempo.rds")
write_rds(pred_h_personas, "pred_h_personas.rds")





Simulación según último valor

 ■                                  1% |  ETA: 14m

 ■                                  1% |  ETA: 12m

 ■■                                 2% |  ETA: 11m

 ■■                                 2% |  ETA: 11m

 ■■                                 3% |  ETA: 11m

 ■■                                 4% |  ETA: 10m

 ■■                                 4% |  ETA: 10m

Warning message in rpois(1, pars$lambda):
“NAs produced”
 ■■■                                5% |  ETA: 10m

 ■■■                                6% |  ETA: 10m

 ■■■                                6% |  ETA: 10m

 ■■■                                7% |  ETA: 10m

 ■■■                                8% |  ETA: 10m

 ■■■                                8% |  ETA: 10m

 ■■■■                               9% |  ETA: 10m

 ■■■■                               9% |  ETA: 10m

Warning message in rpois(1, pars$lambda):
“NAs produced”
 ■■■■                              10% |  ETA: 10m

 ■■■■                  